In [1]:
import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
import SimpleITK as sitk # para calcular rangos
import numpy as np
import io
from PIL import Image
import random
import sys
import pandas as pd
import math
import pandas as pd
import sklearn
import sklearn.metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [2]:
sys.path.append("/Users/hugom/PET-IA/src")

In [3]:
from datasets import BaseDataset
from transforms import TransformGridImage, ToLabelOutputConfigurable
from train_lib import train_model, set_parameter_requires_grad, initialize_model
from util import collectAllData
import torch
import torchvision
from torchvision import transforms, utils, models, datasets

In [4]:
mean = 2300.842041015625
std = 4192.4441558594435

In [5]:
config = {
  "last_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_3classes_0_epoch21.pth",
    "num_classes": 3
  },
  "sev_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_3classes_0_epoch31.pth",
    "num_classes": 3
  },
  "visit_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_3classes_0_epoch10.pth",
    "num_classes": 3
  },
  "last_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_2classes_0_epoch18.pth",
    "num_classes": 2
  },
  "sev_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_2classes_0_epoch16.pth",
    "num_classes": 2
  },
  "visit_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_2classes_0_epoch5.pth",
    "num_classes": 2
  }
}


In [6]:
def executeModel(dataloader, weights, num_classes):
    featureExtract = False
    dropoutRate = 0.6
    auxEnabled = True
    usePretrained = False
    model_ft, input_size = initialize_model('inception', num_classes, featureExtract, dropoutRate, auxEnabled, use_pretrained=usePretrained)
    device = torch.device(deviceName if torch.cuda.is_available() else "cpu")
    model_state_dict = torch.load(weights, map_location=device)
    model_ft.load_state_dict(model_state_dict)

    model_ft.to(device)
    
    testY, predY = collectAllData(dataloader, model_ft, device, num_classes)
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(testY)
    print(preds)
    
    print(f"Accuracy: {acc}")
    print(f"B acc: {bacc}")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    
    print("Confusion matrix:")
    print(matrix)
    
    if (num_classes == 2):
        #lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
        #print(f"AUC ROC: {lr_auc}")
        return
    
    print("------------")
    print("Merging 2 with 0")
    
    predY[:, 0] = predY[:, 0] + predY[:, 2] #sumo MCI con CN
    predY = predY[:, 0:2] # Borro MCI
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(f"Accuracy merged: {acc}")
    print(f"B acc merged: {bacc}")
    
    print("Confusion matrix merged:")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    print(matrix)
    
    #lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
    #print(f"AUC ROC: {lr_auc}")

In [7]:
tsvFile = '/Users/hugom/PET-IA/Full-DBs/ines-merida-db/pool/DM/TEP/CERMEP_MXFDG/BASE/DATABASE_SENT/ALL/participants.tsv'
imagesFolder = '/Users/hugom/Tesis/Imagenes/merida-preprocessed'
meridaTransforms = torchvision.transforms.Compose([
    TransformGridImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([mean, mean, mean], [std, std, std])
])
dicti = {
    "AD": 1,
    "non-AD": 0
}
csv = pd.read_table(tsvFile, index_col = False)
csv['Group'] = 'non-AD'
dataset = BaseDataset('merida', csv, imagesFolder, studyIDLabel = 'participant_id', transform = meridaTransforms, target_transform = ToLabelOutputConfigurable(dicti), truthLabel = 'Group')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [8]:
for key in config:
    print(f"Ejecutando {key}")
    executeModel(dataloader, **config[key])
    print("#################")

Ejecutando last_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048


/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/si

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Accuracy: 0.13513513513513514
B acc: 0.13513513513513514
Confusion matrix:
[[ 5 32]
 [ 0  0]]
------------
Merging 2 with 0
Accuracy merged: 0.13513513513513514
B acc merged: 0.13513513513513514
Confusion matrix merged:
[[ 5 32]
 [ 0  0]]
#################
Ejecutando sev_3_classes
num featurs2048


...

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[2 2 1 1 0 1 2 2 2 1 2 2 2 2 2 1 2 2 2 1 2 2 2 1 1 2 2 1 1 2 2 2 2 2 2 1 2]
Accuracy: 0.02702702702702703
B acc: 0.02702702702702703
Confusion matrix:
[[ 1 11 25]
 [ 0  0  0]
 [ 0  0  0]]
------------
Merging 2 with 0
Accuracy merged: 0.9459459459459459
B acc merged: 0.9459459459459459
Confusion matrix merged:
[[35  2]
 [ 0  0]]
#################
Ejecutando last_2classes
num featurs2048
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 1.0
B acc: 1.0
Confusion matrix:
[[37]]
#################
Ejecutando sev_2classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048


/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (whic

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Accuracy: 0.16216216216216217
B acc: 0.16216216216216217
Confusion matrix:
[[ 6 31]
 [ 0  0]]
#################
Ejecutando visit_2classes
num featurs2048
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 1.0
B acc: 1.0
Confusion matrix:
[[37]]
#################


# Nuevos

In [9]:
configNew = {
  "last_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_CN,AD_0_epoch42.pth",
    "num_classes": 2
  },
  "sev_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_CN,AD_0_epoch12.pth",
    "num_classes": 2
  },
  "visit_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_CN,AD_0_epoch19.pth",
    "num_classes": 2
  },
  "last_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_MCI,AD_0_epoch45.pth",
    "num_classes": 2
  },
  "sev_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_MCI,AD_0_epoch17.pth",
    "num_classes": 2
  },
  "visit_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_MCI,AD_0_epoch1.pth",
    "num_classes": 2
  }
}

In [10]:
dataset = BaseDataset('merida', csv, imagesFolder, studyIDLabel = 'participant_id', transform = meridaTransforms, target_transform = ToLabelOutputConfigurable(dicti), truthLabel = 'Group')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [11]:
for key in configNew:
    print(f"Ejecutando {key}")
    executeModel(dataloader, **configNew[key])
    print("#################")

Ejecutando last_cn_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048


/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (whic

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0]
Accuracy: 0.2702702702702703
B acc: 0.2702702702702703
Confusion matrix:
[[10 27]
 [ 0  0]]
#################
Ejecutando sev_cn_vs_ad
num featurs2048


...

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0]
Accuracy: 0.35135135135135137
B acc: 0.35135135135135137
Confusion matrix:
[[13 24]
 [ 0  0]]
#################
Ejecutando last_mci_vs_ad
num featurs2048


/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (whic

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.918918918918919
B acc: 0.918918918918919
Confusion matrix:
[[34  3]
 [ 0  0]]
#################
Ejecutando sev_mci_vs_ad
num featurs2048


/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (whic

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.972972972972973
B acc: 0.972972972972973
Confusion matrix:
[[36  1]
 [ 0  0]]
#################
Ejecutando visit_mci_vs_ad
num featurs2048
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 1.0
B acc: 1.0
Confusion matrix:
[[37]]
#################
